In [3]:
!pip install -q -U google-genai


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
f = open("gemini_api.txt")
api_key = f.read()

In [5]:
from google import genai
from pydantic import BaseModel

client = genai.Client(api_key=api_key)

class Result(BaseModel):
    object_presence: bool
    correct_object_position: bool

def get_object_presence(image_path, subject):
    # Upload the image
    rgb_image = image_path
    uploaded_file1 = client.files.upload(file=rgb_image)

    subject = subject

    # Create the prompt with text and multiple images
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            f"The left image is the RGB picture of a scene. The right image is the voxel similarity map corresponding to this scene and the object '{subject}'. Yellow means high similarity (1) and purple means low similarity(0). Is there a {subject} in the RGB image ? Does the similarity map show higher values at the position of this object ?",
            uploaded_file1, 
            #uploaded_file2,
        ],
        config={
            "response_mime_type": "application/json",
            "response_schema": list[Result],
        }
    )

    return response.parsed

print(get_object_presence("E_grade_pics/sofa/sofa_18.png", "sofa"))

[Result(object_presence=True, correct_object_position=True)]
